In [1]:
# Mike's dial plot function

class Dial:
    def __init__(self, ax, angular_bins, radial_bins, H):
        """ 
        This class makes a dial (polar) plot were MLT is the azimuthal
        coordinate and L shell is the radial coordinate. 
        """
        self.ax = ax
        self.angular_bins = angular_bins
        self.radial_bins = radial_bins
        self.H = H

        if 'Polar' not in str(type(ax)):
            raise ValueError('Subplot is not polar. For example, '
                'create ax with \n ax[0] = plt.subplot(121, projection="polar")')
        return

    def draw_dial(self, cb_label, colorbar=True, L_labels=[2,4,6,8], cb_ticksize=15, cb_labelsize=15, mesh_kwargs={}, colorbar_kwargs={}):
        """
        Draws a dial plot on the self.ax subplot object (must have projection='polar' kwarg). 
        colorbar=True - Plot the colorbar or not.
        L_labels=[2,4,6,8] - What L labels to plot
        mesh_kwargs={} - The dictionary of kwargs passed to plt.pcolormesh() 
        colorbar_kwargs={} - The dictionary of kwargs passed into plt.colorbar()
        """
        self.L_labels = L_labels

        angular_grid, radial_grid = np.meshgrid(self.angular_bins, self.radial_bins)

        # Try-except block deals with the dimensions of the mesh and taransposes it
        # if necessary.
        try:
            p = self.ax.pcolormesh(angular_grid*np.pi/12, radial_grid, self.H.T, **mesh_kwargs)
        except TypeError as err:
            if 'Dimensions of C' in str(err):
                p = self.ax.pcolormesh(angular_grid*np.pi/12, radial_grid, self.H, **mesh_kwargs)
            else:
                raise

        self.draw_earth()
        self._plot_params()

        if colorbar:
            cb = plt.colorbar(p, ax=self.ax, **colorbar_kwargs)
            cb.set_label(label=cb_label, size=cb_labelsize)
            cb.ax.tick_params(labelsize=cb_ticksize)
        return

    def draw_earth(self, earth_resolution=50):
        """ Given a subplot object, draws the Earth with a shadow"""
        # Just x,y coords for a line (to map to polar coords)
        earth_circ = (np.linspace(0, 2*np.pi, earth_resolution), np.ones(earth_resolution)) 
        # x, y_lower, y_upper coords for Earth's shadow (maps to polar).
        earth_shadow = (
                        np.linspace(-np.pi/2, np.pi/2, earth_resolution), 
                        0, 
                        np.ones(earth_resolution)
                        )
        self.ax.plot(*earth_circ, c='k')
        self.ax.fill_between(*earth_shadow, color='k')
        return

    def _plot_params(self):
        # Draw L shell contours and get L and MLT labels 
        L_labels_names = self._draw_L_contours()
        mlt_labels = (self.ax.get_xticks()*12/np.pi).astype(int)
        # Sun facing up.
        self.ax.set_xlabel('MLT')
        self.ax.set_theta_zero_location("S") # Midnight at bottom
        self.ax.set_xticklabels(mlt_labels) # Transform back from 0->2pi to 0->24.
        self.ax.set_yticks(self.L_labels)
        self.ax.set_yticklabels(L_labels_names, fontdict={'horizontalalignment':'right'})
        return

    def _draw_L_contours(self, earth_resolution=50):
        """ Plots a subset of the L shell contours. """
        # Draw azimuthal lines for a subset of L shells.
        L_labels_names = [str(i) for i in self.L_labels[:-1]] + [f'L = {self.L_labels[-1]}']
        # L_labels_names = [str(i) for i in self.L_labels]
        for L in self.L_labels:
            self.ax.plot(np.linspace(0, 2*np.pi, earth_resolution), 
                        L*np.ones(earth_resolution), ls=':', c='k')
        return L_labels_names

In [ ]:
# example of Dial plot code

# create MB iso vs MLT/LShell dial plot
# choose colorbar
cmap = 'magma'

# create empty histogram
H_isoavgs_MB = np.nan*np.zeros((len(MLT_bins), len(L_bins), len(percentiles)))

# populate histogram
for i, (start_MLT, end_MLT) in enumerate(zip(MLT_bins[:-1], MLT_bins[1:])):
    for j, (start_L, end_L) in enumerate(zip(L_bins[:-1], L_bins[1:])):
        
        iso_values = iso_list_MB[(MLT_list_MB > start_MLT) & (MLT_list_MB < end_MLT) & (LS_list_MB > start_L) & \
                                 (LS_list_MB < end_L)]

        if iso_values.shape[0] >= statistics_thresh:
            H_isoavgs_MB[i, j, :] = np.nanpercentile(iso_values, percentiles)

# plot histograms
fig = plt.figure(figsize=(10, 8))
ax = [plt.subplot(2, 2, i, projection='polar') for i in range(1, 5)]
cb_label = 'Isotropy Index'

for i, ax_i in enumerate(ax[:-1]):
    d = Dial(ax_i, MLT_bins, L_bins, H_isoavgs_MB[:, :, i])
    d.draw_dial(cb_label=cb_label,
                L_labels=L_labels,
                mesh_kwargs={'cmap':cmap, 'vmin': 0, 'vmax': 1},
                colorbar_kwargs={'label':cb_label, 'pad':0.1})
    annotate_str = f'({string.ascii_lowercase[i]}) {percentiles[i]}th percentile'
    ax_i.text(-0.2, 1.2, annotate_str, va='top', transform=ax_i.transAxes, 
            weight='bold', fontsize=15)

# choose colorbar
cmap = 'viridis'
cb_label = 'Number of microbursts'

# plot MB occurance histogram
d4 = Dial(ax[-1], MLT_bins, L_bins, H_LvMLT_MB)
d4.draw_dial(cb_label=cb_label,
             L_labels=L_labels,
             mesh_kwargs={'norm':matplotlib.colors.LogNorm(), 'cmap':cmap},
             colorbar_kwargs={'label':cb_label, 'pad':0.1})
annotate_str = f'({string.ascii_lowercase[len(ax)-1]}) Microburst occurrence'
ax[-1].text(-0.2, 1.2, annotate_str, va='top', transform=ax[-1].transAxes, 
            weight='bold', fontsize=15)

for ax_i, label_i in zip(ax, string.ascii_lowercase):
    annotate_str = f'({label_i})'
    ax_i.text(0, 1, annotate_str, va='top', color='white', weight='bold', 
                transform=ax_i.transAxes, fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
# label fix

# change MLT labels
d.ax.set_xticklabels(np.array([0,3,6,9,12,15,18,21,24]), fontsize=15)
d.ax.set_xlabel('MLT', fontsize=18)
# change L labels
d.ax.set_yticklabels(np.array(['2','4','L=6']),fontsize=15)
d.ax.set_ylabel('', fontsize=18)